In [2]:
!pip3 install pyro-ppl

In [3]:
import numpy as np
import pandas as pd

import os
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
import random as rnd
rnd.seed(100)

import matplotlib.pyplot as plt
import seaborn as sns

import gresearch_crypto

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.random.set_seed(200)
print('tf version:', tf.__version__)

# Check GPU Availability in Tensorflow
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

# List Devices including GPU's with Tensorflow
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# Check GPU in Tensorflow
tf.test.is_gpu_available()
    

FOLDER = os.path.join(os.getcwd(), 'dev')
if not os.path.isdir(FOLDER):
    os.mkdir(FOLDER)
    print('created', FOLDER)

In [4]:
def weighted_correlation(a, b, weights):
    
    w = np.ravel(weights)
    a = np.ravel(a)
    b = np.ravel(b)
    
    sum_w = np.sum(w)
    mean_a = np.sum(a * w) / sum_w
    mean_b = np.sum(b * w) / sum_w
    var_a = np.sum(w * np.square(a - mean_a)) / sum_w
    var_b = np.sum(w * np.square(b - mean_b)) / sum_w
    
    cov = np.sum((a * b * w)) / np.sum(w) - mean_a * mean_b
    corr = cov / np.sqrt(var_a * var_b)
    
    return corr

In [5]:
dtype = {'Asset_ID': 'int8', 'Weight': float, 'Asset_Name': str}
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv', dtype=dtype)
asset_details = asset_details.sort_values(by=['Asset_ID']).reset_index(drop=True)
weights = asset_details['Weight'].values.astype('float32')
asset_details

In [6]:
dtype = {'timestamp': 'int64', 'Asset_ID': 'int8', 'Count': 'int32', 
         'Open': 'float64', 'High': 'float64', 'Low': 'float64', 'Close': 'float64',
         'Volume': 'float64', 'VWAP': 'float64', 'Target': 'float64'}

df = pd.DataFrame()
for fname in ['train.csv', 'supplemental_train.csv']:
    df = df.append(pd.read_csv(f'../input/g-research-crypto-forecasting/{fname}', low_memory=False, dtype=dtype))

dt = df['timestamp'].astype('datetime64[s]')
print([dt.min(), dt.max()])
del dt

print(df.shape)
df.head()

In [7]:
def prep_asset(df_):
    df = df_.drop('Asset_ID', axis=1).copy()
    df['datetime'] = df['timestamp'].astype('datetime64[s]')
    df = df.set_index('datetime')    
    df = df.sort_index()
    df = df.loc[~df.index.duplicated()]
    df[df.isin([np.inf, -np.inf])] = np.nan
    df = df.reindex( pd.date_range(start=df.index.min(), end=df.index.max(), freq='min') )
    df = df.interpolate(method='linear', limit_direction='both', axis=0)
    return df

dfs = OrderedDict([(i, prep_asset(df[df['Asset_ID']==i])) for i in range(14)])
del df
dfs_prod = {i: df.loc[[df.index[-1]]].copy() for i, df in dfs.items()}

dfs[0].head()

In [8]:
def time_bounds(dfs, left_shift=0, right_shift=0):
    start = max([df.index.min() for df in dfs.values()]) + relativedelta(months=left_shift)
    end = min([df.index.max() for df in dfs.values()]) - relativedelta(months=right_shift)
    return start, end

def trim_dfs(dfs):
    start, end = time_bounds(dfs)
    print(start, end)
    for i, df in dfs.items():
        dfs[i] = df[start:]

trim_dfs(dfs)

In [9]:
def enrich(df, suffix=''):
    df['Upper_Shadow'+suffix] = df['High'].values / np.maximum(df['Close'].values, df['Open'].values)
    df['Lower_Shadow'+suffix] =  np.minimum(df['Close'].values, df['Open'].values) / df['Low'].values
    df['spread'+suffix] = df['High'].values / df['Low'].values
    #df['mean_trade'+suffix] = df['Volume'].values/df['Count'].values
    df['price_change'+suffix] = df['Close'].values/df['Open'].values    

def enrichment(dfs):   
    for i in range(14):
        enrich(dfs[i])
        
enrichment(dfs)

dfs[1].head(3)

In [10]:
def treat_trend(dfs):
    for i in range(14):
        df = dfs[i]
        for col in ['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']: #, 'mean_trade'
            s = df[col]
            s_trans = s/np.maximum(s.shift(1), 1e-5)
            df[col] = s_trans
        dfs[i] = df.dropna()

treat_trend(dfs)
dfs[1].head(3)

In [11]:
def treat_outliers(dfs):
    bounds = {}
    columns = ['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Target'] #, 'mean_trade'
    columns_q25 = [col+'_q25' for col in columns]
    columns_q75 = [col+'_q75' for col in columns]
    columns_join = ['month','hour']
    
    for i in range(14):
        df = dfs[i]
        
        df_q25 = df[columns].groupby([df.index.month, df.index.hour]).agg(lambda x: np.percentile(x, 25))   
        df_q25 = df_q25.add_suffix('_q25')
        df_q25[columns_join[0]] = df_q25.index.get_level_values(0)
        df_q25[columns_join[1]] = df_q25.index.get_level_values(1)
        
        df_q75 = df[columns].groupby([df.index.month, df.index.hour]).agg(lambda x: np.percentile(x, 75))   
        df_q75 = df_q75.add_suffix('_q75')
        df_q75[columns_join[0]] = df_q75.index.get_level_values(0)
        df_q75[columns_join[1]] = df_q75.index.get_level_values(1)
        
        df[columns_join[0]] = df.index.month
        df[columns_join[1]] = df.index.hour
        
        df = df.merge(df_q25, on=columns_join, how='left').merge(df_q75, on=columns_join, how='left')
        
        q25 = df[columns_q25].values
        q75 = df[columns_q75].values
        iqr = q75 - q25
        cut_off = iqr * 3
        lb = q25 - cut_off
        ub = q75 + cut_off
        
        df.loc[:,columns] = np.maximum(lb, np.minimum(df.loc[:,columns].values, ub))
            
        df = df.drop(columns_q25+columns_q75+columns_join, axis=1)
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
        df = df.set_index('datetime')  
        dfs[i] = df
    
        bounds['lb_{}'.format(i)] = lb
        bounds['ub_{}'.format(i)] = ub
        
    return bounds

bounds = treat_outliers(dfs)

In [12]:
def smoothing(dfs):
    for i in range(14):
        dfs[i]['Target'] = dfs[i]['Target'].ewm(span=5).mean()

smoothing(dfs)

In [13]:
def target_scaler(dfs):    
    from sklearn.preprocessing import MinMaxScaler
    tscalers = {}
    for i in range(14):  
        tscalers[i] = MinMaxScaler(feature_range=(-1, 1))
        values = dfs[i]['Target'].values.reshape(-1,1)        
        dfs[i]['Target'] = tscalers[i].fit_transform(values)
    return tscalers

tscalers = target_scaler(dfs)

In [14]:
plt.figure(figsize=(20,5))
dfs[1][:100000].Target.plot()
#plt.ylabel('value')
#plt.xlabel('data')

"""
for col in dfs[1].iloc[:,1:]:
    plt.figure()
    dfs[1][col].plot()
    plt.title(col)"""

In [15]:
def add_lagged(dfs):
    for i, df in dfs.items():
        df_lag = df.drop(['timestamp', 'Target'], axis=1).shift(periods=1).add_suffix('_lag1')
        dfs[i] =  pd.concat([df, df_lag], axis=1).dropna()

add_lagged(dfs)
dfs[1].head(3)

In [16]:
#dfs[1].groupby([dfs[1].index.year, dfs[1].index.month])['Target'].mean().plot()
#pd.Timestamp(1514764860, unit='s').weekday()

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.preprocessing import FunctionTransformer

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer


def timestamp_func(df):
    
    max_month = 12
    max_weekday = 6
    max_hour = 23
    max_minute = 59    
    max_hour_minute = max_hour*max_minute + max_minute

    month = df.index.month
    weekday = df.index.weekday
    hour_minute = df.index.hour*max_minute + df.index.minute
    
    arr = np.zeros((len(df), 6), dtype='float32')
    # month
    arr[:,0] = np.sin(month * (2 * np.pi / max_month))
    arr[:,1] = np.cos(month * (2 * np.pi / max_month))
    # weekday
    arr[:,2] = np.sin(weekday * (2 * np.pi / max_weekday))
    arr[:,3] = np.cos(weekday * (2 * np.pi / max_weekday))
    # hour_minute
    arr[:,4] = np.sin(hour_minute * (2 * np.pi / max_hour_minute))
    arr[:,5] = np.cos(hour_minute * (2 * np.pi / max_hour_minute))
    
    return arr

columns = ['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']
columns += ['Upper_Shadow', 'Lower_Shadow', 'spread', 'price_change'] #'mean_trade', 
columns += [col+'_lag1' for col in columns]

pipelines = {'time': FunctionTransformer(func=lambda x: timestamp_func(x))}
for i, df in dfs.items():
    extractor = FunctionTransformer(func=lambda x: x[columns].values)
    scaler = StandardScaler()
    pipe = make_pipeline(extractor, scaler)
    pipelines[i] = pipe.fit(df)

n_cyclic = timestamp_func(dfs[0]).shape[1]
n_features = pipelines[0].transform(dfs[0]).shape[1]

In [18]:
def data_generator(dfs, start, end, pipelines, batch_size=256, shuffle=True, epochs=1):
    """A data generator function. Adapted for single asset"""    
    
    # create an array with the indexes that can be shuffled
    indexes = dfs[0][start:end].index.tolist()
    length = len(indexes)
    
    # shuffle the indexes
    if shuffle:
        rnd.shuffle(indexes)
    
    # init 
    idx = 0 # current location
    batch_indexes = [0] * batch_size    
    epoch = 0
    flag = False
    
    targets = np.zeros((batch_size,1), dtype='float32')
    features = np.zeros((batch_size, n_features), dtype='float32')
    time_encoding = np.zeros((batch_size, n_cyclic), dtype='float32')
    #targets = np.zeros((batch_size,14), dtype='float32')
    #features = np.zeros((batch_size,14,n_features), dtype='float32')
    #time_encoding = np.zeros((batch_size,14,n_cyclic), dtype='float32') 
    #asset_encoding = np.tile(np.expand_dims(np.eye(14, dtype='float32'), axis=0), (batch_size,1,1))
    #encodings = np.concatenate([time_encoding, asset_encoding], axis=-1)
    
    while True:
        
        if flag:
            break
        
        for i in range(batch_size):
            if idx >= length:
                epoch += 1
                flag = epoch>=epochs # determine if continue after pass through data
                idx = 0
                if shuffle:
                    rnd.shuffle(indexes)                    
            batch_indexes[i] = indexes[idx]            
            idx += 1     
        
        i = 1
        df_batch = dfs[i].loc[batch_indexes]
        features = pipelines[i].transform(df_batch)
        time_encoding = pipelines['time'].transform(df_batch)  
               
        inputs = np.concatenate([time_encoding, features], axis=-1)
        targets = df_batch['Target'].values#.reshape(-1,1)
        
        yield torch.tensor(inputs, dtype=torch.float), torch.tensor(targets, dtype=torch.float)
        '''
        for i, df in dfs.items():
            df_batch = df.loc[batch_indexes]
            features[:,i,:] = pipelines[i].transform(df_batch)
            encodings[:,i,:n_cyclic] = pipelines['time'].transform(df_batch)            
            targets[:,i] = df_batch['Target']
        
        inputs = [features, encodings]'''
                
        #yield inputs, targets

# testing
#start, end = time_bounds(dfs)
#generator = data_generator(dfs, start, end, pipelines)
#inputs, targets = next(generator)

In [32]:
import torch
from torch.distributions import constraints

import pyro
import pyro.distributions as pdist

from pyro.nn import PyroModule, PyroSample
import torch.nn as nn

In [33]:
def Model(x_dim, y_dim, h_dims=[256, 64]):

    def wrapper(x, y=None):
        
        # standard deviation of Normals        
        sd = 1
        
        # Layer 1
        w1 = pyro.sample("w1", pdist.Normal(0, sd).expand([x_dim, h_dims[0]]).to_event(2))
        b1 = pyro.sample("b1", pdist.Normal(0, sd).expand([h_dims[0]]).to_event(1))
        # Layer 2
        w2 = pyro.sample("w2", pdist.Normal(0, sd).expand([h_dims[0], h_dims[1]]).to_event(2))
        b2 = pyro.sample("b2", pdist.Normal(0, sd).expand([h_dims[1]]).to_event(1))
        # Layer 3
        w3 = pyro.sample("w3", pdist.Normal(0, sd).expand([h_dims[1], y_dim]).to_event(2))
        b3 = pyro.sample("b3", pdist.Normal(0, sd).expand([y_dim]).to_event(1))       
        
        # NN
        h1 = torch.tanh((x @ w1) + b1)
        h2 = torch.tanh((h1 @ w2) + b2)
        mean = (h2 @ w3) + b3
        
        sigma = pyro.sample("sigma", pdist.Uniform(0.,1))
        
        with pyro.plate("targets", x.shape[0]):
            obs = pyro.sample("obs", pdist.Normal(mean.squeeze(), sigma), obs=y)
        
    return wrapper

'''
class Model(PyroModule):
    # https://www.kaggle.com/carlossouza/simple-bayesian-neural-network-in-pyro
    def __init__(self, x_dim, h1=128, h2=32):
        super().__init__()
        self.fc1 = PyroModule[nn.Linear](x_dim, h1)
        self.fc1.weight = PyroSample(pdist.Normal(0., 1.).expand([h1, x_dim]).to_event(2))
        self.fc1.bias = PyroSample(pdist.Normal(0., 1.).expand([h1]).to_event(1))
        
        self.fc2 = PyroModule[nn.Linear](h1, h2)
        self.fc2.weight = PyroSample(pdist.Normal(0., 1.).expand([h2, h1]).to_event(2))
        self.fc2.bias = PyroSample(pdist.Normal(0., 1.).expand([h2]).to_event(1))
        
        self.fc3 = PyroModule[nn.Linear](h2, 1)
        self.fc3.weight = PyroSample(pdist.Normal(0., 1.).expand([1, h2]).to_event(2))
        self.fc3.bias = PyroSample(pdist.Normal(0., 1.).expand([1]).to_event(1))
        self.relu = nn.ReLU()

    def forward(self, x, y=None):
        #x = x.reshape(-1, 1)
        h1 = self.relu(self.fc1(x))
        h2 = self.relu(self.fc2(h1))
        mu = self.fc3(h2).squeeze()
        sigma = pyro.sample("sigma", pdist.Uniform(0., 1.))
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", pdist.Normal(mu, sigma), obs=y)
        return mu'''
    
pyro.clear_param_store()
# Instantiate the Model object
model = Model(x_dim=n_cyclic+n_features, y_dim=1)
#model = Model(x_dim=n_cyclic+n_features)
guide = pyro.infer.autoguide.AutoDelta(model)

In [34]:
#model.predict([np.random.randn(1, 14, n_features), np.random.randn(1, 14, n_cyclic+14)]).ravel()

In [35]:
def train_model(model, guide):
    
    train_start, train_end = time_bounds(dfs, left_shift=0) #, right_shift=3
    _, val_end = time_bounds(dfs)

    #train_generator = data_generator(dfs, train_start, train_end, pipelines, epochs=10)
    #val_generator = data_generator(dfs, train_end, val_end, pipelines)
    
    # SVI    
    svi = pyro.infer.SVI(model,
                         guide,
                         pyro.optim.Adam({"lr": 1e-3}),
                         loss=pyro.infer.Trace_ELBO())

    # Clear any previously used parameters
    pyro.clear_param_store()
    
    train_n = train_loss = 0
    val_n = val_loss = 0
    tic = time.time()    
    
    history = {'train_loss': [], 'val_loss': []}
    epochs = 10
    step = 0
        
    for epoch in range(1,epochs+1):     
        
        train_generator = data_generator(dfs, train_start, train_end, pipelines, batch_size=256, shuffle=True)
        
        while True:
            
            step += 1
            
            try:
                batch_inputs, batch_targets = next(train_generator)
            except:
                break            
            
            #train_loss += model.train_on_batch(batch_inputs, batch_targets)
            train_loss += svi.step(batch_inputs, batch_targets)
            train_n += 1
            
            if False and step%5000==0:
                lr = model.optimizer.learning_rate.numpy()*0.75
                model.optimizer.learning_rate.assign(lr)            

            if step%500 == 0:
                train_loss = np.sqrt(train_loss / train_n)
                print(f'step: {step} ------------------------')
                print('train_loss: {:.4f} train_time: {}'.format(train_loss, round((time.time()-tic)/60))) 
                history['train_loss'].append(train_loss)
                train_n = train_loss = 0
                tic = time.time()

            if False and step%1000 == 0:            
                val_generator = data_generator(dfs, train_end, val_end, pipelines, shuffle=False)
                while True:
                    try:
                        batch_inputs, batch_targets = next(val_generator)
                        batch_predictions = model(batch_inputs)
                        loss = custom_loss(batch_targets, batch_predictions, w)
                        val_n += 1
                        val_loss += loss.numpy()
                    except:
                        break
                val_loss = np.sqrt(val_loss / val_n)
                print('val_loss: {:.4f} val_time: {}'.format(val_loss, round((time.time()-tic)/60)))           
                history['val_loss'].append(val_loss)
                val_n = val_loss = 0
                tic = time.time()
            
    return history
    
history = train_model(model, guide)

In [36]:
train_start, train_end = time_bounds(dfs, right_shift=3)
_, val_end = time_bounds(dfs)

generator = data_generator(dfs, train_end, val_end, pipelines, shuffle=False, batch_size=1024) # val
x_test, y_test = next(generator)
y_true = y_test.detach().numpy().ravel()

# Get the posterior predictive distribution by sampling the model's parameters from the Guide object and applying the model to the test set.
guide.requires_grad_(False)

predictive = pyro.infer.Predictive(model, guide=guide, num_samples=100, return_sites=["obs"])
preds = predictive(x_test)

samples = preds['obs']

from pyro.ops.stats import quantile
p10, p50, p90 = quantile(samples, (0.1, 0.5, 0.9)).squeeze(-1)
p10, p50, p90 = p10.detach().numpy().ravel(), p50.detach().numpy().ravel(), p90.detach().numpy().ravel()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(y_true, 'o', markersize=1)
ax.plot(p50)
ax.fill_between(y_true, p10, p90, alpha=0.5, color='#ffcd3c');

In [37]:
samples.shape

In [38]:
#model.save(f'{FOLDER}/model')
#model = tf.keras.models.load_model(f'{FOLDER}/model')

In [39]:
def show_performance():
    train_start, train_end = time_bounds(dfs, right_shift=3)
    _, val_end = time_bounds(dfs)

    #generator = data_generator(dfs, train_start, train_end, pipelines, shuffle=False) # train data
    generator = data_generator(dfs, train_end, val_end, pipelines, shuffle=False, batch_size=1024) # val

    batch_inputs, batch_targets = next(generator)
    batch_predictions = model.predict(batch_inputs)

    i = 1
    y_pred = batch_predictions[:,i]
    y_true = batch_targets[:,i]
    
    plt.figure(figsize=(20,10))
    #plt.plot(y_true-y_pred)
    plt.plot(y_pred)
    plt.plot(y_true)
    plt.title('model performance')
    plt.ylabel('value')
    plt.xlabel('data')
    plt.legend(['y_pred', 'y_true'], loc='upper right')
    plt.show()

show_performance()

In [ ]:
"""
plt.plot(history['train_loss'])
plt.plot(history['val_loss'])
plt.title('evolution of loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'], loc='upper right')
plt.show()"""